<a href="https://colab.research.google.com/github/JoanChege/CHATBOT1/blob/main/Chatbottrial8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. IMPORT EVERYTHING
2.   SET JSON FILE AS PANDAS DATAFRAME
3.   PREPROCESS THE DATA
4.   SPLIT DATA INTO TRAINING AND TESTING DATA





In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import SGD
#from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import re
import nltk
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
#import the dataset
with open('/content/dataset3.json') as f:
    intents = json.load(f)




# Preprocess Data-remove stopwords
stop_words = set(stopwords.words('english'))

data = []
labels = []

#perform tokenization, filtering, lemmatizatiopn
if intents is not None:
  for intent in intents['intents']:
    for pattern in intent['patterns']:
        pattern = pattern.strip() # Remove leading/trailing whitespace
        tokenized_pattern = nltk.word_tokenize(pattern)
        filtered_pattern = [word.lower() for word in tokenized_pattern if word.isalnum() and not word in stop_words]
        lemmatizer = WordNetLemmatizer()
        lemmatized_pattern = lemmatized_pattern = [lemmatizer.lemmatize(word) for word in filtered_pattern]
        data.append(' '.join(lemmatized_pattern))
        labels.append(intent['tag'])
#The data list contains all the patterns, and the labels list contains all the corresponding intent tags.
# Create a Pandas DataFrame
df = pd.DataFrame({'Patterns': data, 'Tags': labels})

# Encode the Labels by converting tags into numerical form
encoder = LabelEncoder()
df['Tags'] = encoder.fit_transform(df['Tags'])
df['Tags'] = to_categorical(df['Tags'])

The encoding of the labels: The output of this code will be a DataFrame where the 'Tags' column has been converted into one-hot encoded binary values. This is useful for feeding the data into a neural network for text classification, where each class of text can be represented as a one-hot encoded vector.




In [ ]:
df

In [7]:
import collections
word_counts = collections.Counter(data)
num_words = len(word_counts)

print(f"The number of unique words in the vocabulary is {num_words}")

The number of unique words in the vocabulary is 3844


PERFORM FEATURE EXTRACTION USING TDIF

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
#The CountVectorizer converts the text data into numerical vectors by counting the occurrence of each word in the text
vectorizer = CountVectorizer()
vectorizer.fit(data)

CountVectorizer()

In [9]:
X = vectorizer.transform(data) # transform text to feature vectors

In [ ]:
train_ratio = 0.8 #80%
val_ratio = 0.1 #10%
test_ratio = 0.1 #10%

In [ ]:
X, y = shuffle(X, y)

In [ ]:
total = len(X)

train_size = int(train_ratio * total)
val_size = int(val_ratio * total)
test_size = int(test_ratio * total)

In [10]:
# Encode labels as unique intergers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(labels)
#y_train = np.array(labels)


In [12]:
print(X.shape[0])
print(len(y_train))

3974
3974


In [ ]:
# Split data
X_train, X_val, X_test, y_train, y_val, y_test = train_test_split(X, y, test_size=0.2)


In [13]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(X_train)

In [14]:
print(X_train.shape[0])
print(len(y_train))

3179
3179


In [ ]:
print(X_test)

In [ ]:
print(y_train)

In [ ]:
print(y_test)

In [15]:
num_classes = len(le.classes_)

In [ ]:
num_classes = len(le.classes_)
y_train = np.reshape(y_train, (-1,1))

In [16]:
model = Sequential()
model.add(Embedding(len(vectorizer.vocabulary_), 100, input_length=X_train.shape[1]))
model.add(Bidirectional(LSTM(64, return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [17]:
print(y_train.shape)
print(model.predict(X_train).shape)

(3179,)
100/100 [==============================] - 262s 3s/step
(3179, 317)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8660, 100)         866000    
                                                                 
 bidirectional (Bidirection  (None, 128)               84480     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 317)               20605     
                                                        

In [19]:
y_train = y_train.reshape(-1, 1)

In [20]:
num_classes = len(np.unique(y_train))

In [21]:
y_train = np.resize(y_train, (num_classes,))

In [22]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = SGD(learning_rate=0.01,
                            momentum=0.9, #help to improve the training speed and model stability
                            nesterov=True), metrics=['accuracy'])

In [23]:
num_classes = model.layers[-1].units

In [24]:
y_train = np.resize(y_train, (X_train.shape[0],))

In [25]:
print(X_train.shape)
print(y_train.shape)

(3179, 8660)
(3179,)


In [ ]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

In [26]:
model.fit(X_train, y_train, epochs=20,  batch_size=5, verbose=1)

Epoch 1/20
262/636 [===========>..................] - ETA: 2:17:19 - loss: 1.5314 - accuracy: 0.8580

KeyboardInterrupt: ignored